# Assuming 2 sets of point-clouds, we will compute the MMD, Coverage and JSD as done in the paper.

(To compute these metrics you __don't need__ to have tflearn installed, only the structural: EMD, Chamfer losses and sklearn for the JSD.)

In [1]:
%matplotlib inline
%load_ext autoreload
%autoreload 2
import sys
sys.path.insert(0, "/home/gy46/")

In [2]:
import numpy as np
import os.path as osp

from latent_3d_points.src.evaluation_metrics import minimum_mathing_distance, \
jsd_between_point_cloud_sets, coverage

from latent_3d_points.src.in_out import snc_category_to_synth_id,\
                                        load_all_point_clouds_under_folder

Load some point-clouds and make two sets (sample_pcs, ref_pcs) from them. The ref_pcs is considered as the __ground-truth__ data while the sample_pcs corresponds to a set that is matched against it, e.g. comes from a generative model.

In [3]:
# top_in_dir = '../data/shape_net_core_uniform_samples_2048/' # Top-dir of where point-clouds are stored.
# top_in_dir = '../data/ShapeNetV1PCOutput/' # Top-dir of where point-clouds are stored.
top_in_dir = '../data/ShapeNetCore.v2.PC15k/'
class_name = raw_input('Give me the class name (e.g. "chair"): ').lower()
syn_id = snc_category_to_synth_id()[class_name]
class_dir = osp.join(top_in_dir , syn_id, 'val')
# all_pc_data = load_all_point_clouds_under_folder(class_dir, n_threads=8, file_ending='.ply', verbose=True)
all_pc_data = load_all_point_clouds_under_folder(class_dir, n_threads=8, file_ending='.npy', verbose=True, normalize=True)

Give me the class name (e.g. "chair"): airplane
405 pclouds were loaded. They belong in 1 shape-classes.


In [4]:
n_ref = 100 # size of ref_pcs.
n_sam = 150 # size of sample_pcs.
all_ids = np.arange(all_pc_data.num_examples)
print(len(all_ids))
ref_ids = np.random.choice(all_ids, n_ref, replace=False)
sam_ids = np.random.choice(all_ids, n_sam, replace=False)
ref_pcs = all_pc_data.point_clouds[ref_ids][:,:2048,:]
sample_pcs = all_pc_data.point_clouds[sam_ids][:,:2048,:]

405


In [5]:
ref_pcs.shape, sample_pcs.shape

((100, 2048, 3), (150, 2048, 3))

Compute the three metrics.

In [6]:
ae_loss = 'chamfer'  # Which distance to use for the matchings.

if ae_loss == 'emd':
    use_EMD = True
else:
    use_EMD = False  # Will use Chamfer instead.
    
batch_size = 100     # Find appropriate number that fits in GPU.
normalize = True     # Matched distances are divided by the number of 
                     # points of thepoint-clouds.

mmd, matched_dists = minimum_mathing_distance(sample_pcs, ref_pcs, batch_size, normalize=normalize, use_EMD=use_EMD)

cov, matched_ids = coverage(sample_pcs, ref_pcs, batch_size, normalize=normalize, use_EMD=use_EMD)

jsd = jsd_between_point_cloud_sets(sample_pcs, ref_pcs, resolution=28)

/home/gy46/latent_3d_points/src/evaluation_metrics.py:219: UserWarning: Point-clouds are not in unit cube.
  warnings.warn('Point-clouds are not in unit cube.')
/home/gy46/latent_3d_points/src/evaluation_metrics.py:222: UserWarning: Point-clouds are not in unit sphere.
  warnings.warn('Point-clouds are not in unit sphere.')


In [7]:
print mmd, cov, jsd

0.0064208787 0.8 0.013919467510836725


In [8]:
sample_pcs.shape, ref_pcs.shape

((150, 2048, 3), (100, 2048, 3))

In [9]:
from latent_3d_points.src.evaluation_metrics_fast import MMD_COV_EMD_CD
mmd_emd, mmd_cd, cov_emd, cov_cd = MMD_COV_EMD_CD(sample_pcs[:,:2048,:], ref_pcs[:,:2048,:], 
                                                  32, normalize=normalize, verbose=True)
print("MMD-EMD:%s"%mmd_emd)
print("MMD-CD:%s"%mmd_cd)
print("COV-EMD:%s"%cov_emd)
print("COV-CD:%s"%cov_cd)


MMD-COV Loop: 100%|██████████| 100/100 [00:50<00:00,  2.00it/s]

MMD-EMD:0.11683092
MMD-CD:0.0064208787
COV-EMD:0.68
COV-CD:0.66


# For a detailed breakdown of the evaluation functions, inspect their docs.

In [10]:
print coverage.__doc__
print minimum_mathing_distance.__doc__
print jsd_between_point_cloud_sets.__doc__

Computes the Coverage between two sets of point-clouds.

    Args:
        sample_pcs (numpy array SxKx3): the S point-clouds, each of K points that will be matched
            and compared to a set of "reference" point-clouds.
        ref_pcs    (numpy array RxKx3): the R point-clouds, each of K points that constitute the
            set of "reference" point-clouds.
        batch_size (int): specifies how large will the batches be that the compute will use to
            make the comparisons of the sample-vs-ref point-clouds.
        normalize (boolean): if True, the distances are normalized by diving them with
            the number of the points of the point-clouds (n_pc_points).
        use_sqrt  (boolean): When the matching is based on Chamfer (default behavior), if True,
            the Chamfer is computed based on the (not-squared) euclidean distances of the matched
            point-wise euclidean distances.
        sess (tf.Session):  If None, it will make a new Session for th

# Loading the Model

In [11]:
from latent_3d_points.src.ae_templates import mlp_architecture_ala_iclr_18, default_train_params
from latent_3d_points.src.in_out import create_dir
from latent_3d_points.src.autoencoder import Configuration as Conf


n_pc_points = 2048
bneck_size=128
top_out_dir = '../data/'          # Use to save Neural-Net check-points etc.
# experiment_name = 'single_class_ae'
# experiment_name = 'single_class_ae_emd'
# experiment_name = 'emd_ae_chair'
# experiment_name = 'emd_ae_airplane'
experiment_name = 'shapenetcorev2_chamfer_ae_airplane'
# experiment_name = 'chamfer_ae_chair'


In [12]:
train_params = default_train_params()
encoder, decoder, enc_args, dec_args = mlp_architecture_ala_iclr_18(n_pc_points, bneck_size)
train_dir = create_dir(osp.join(top_out_dir, experiment_name))

In [13]:
conf = Conf(n_input = [n_pc_points, 3],
            loss = ae_loss,
            training_epochs = train_params['training_epochs'],
            batch_size = train_params['batch_size'],
            denoising = train_params['denoising'],
            learning_rate = train_params['learning_rate'],
            train_dir = train_dir,
            loss_display_step = train_params['loss_display_step'],
            saver_step = train_params['saver_step'],
            z_rotate = train_params['z_rotate'],
            encoder = encoder,
            decoder = decoder,
            encoder_args = enc_args,
            decoder_args = dec_args
           )
conf.experiment_name = experiment_name
conf.held_out_step = 5   # How often to evaluate根本不需要train什么三千个
# print out loss on 
                         # held_out data (if they are provided in ae.train() ).
conf.save(osp.join(train_dir, 'configuration'))

In [14]:
from latent_3d_points.src.tf_utils import reset_tf_graph
from latent_3d_points.src.point_net_ae import PointNetAutoEncoder

load_pre_trained_ae = True
restore_epoch = 500
if load_pre_trained_ae:
    conf = Conf.load(train_dir + '/configuration')
    reset_tf_graph()
    ae = PointNetAutoEncoder(conf.experiment_name, conf)
    ae.restore_model(conf.train_dir, epoch=restore_epoch)

Building Encoder
('encoder_conv_layer_0', 'conv params = ', 256)
('bnorm params = ', 128)
Tensor("shapenetcorev2_chamfer_ae_airplane_2/Relu:0", shape=(?, 2048, 64), dtype=float32)
('output size:', 131072, '\n')
('encoder_conv_layer_1', 'conv params = ', 8320)
('bnorm params = ', 256)
Tensor("shapenetcorev2_chamfer_ae_airplane_2/Relu_1:0", shape=(?, 2048, 128), dtype=float32)
('output size:', 262144, '\n')
('encoder_conv_layer_2', 'conv params = ', 16512)
('bnorm params = ', 256)
Tensor("shapenetcorev2_chamfer_ae_airplane_2/Relu_2:0", shape=(?, 2048, 128), dtype=float32)
('output size:', 262144, '\n')
('encoder_conv_layer_3', 'conv params = ', 33024)
('bnorm params = ', 512)
Tensor("shapenetcorev2_chamfer_ae_airplane_2/Relu_3:0", shape=(?, 2048, 256), dtype=float32)
('output size:', 524288, '\n')
('encoder_conv_layer_4', 'conv params = ', 32896)
('bnorm params = ', 256)
Tensor("shapenetcorev2_chamfer_ae_airplane_2/Relu_4:0", shape=(?, 2048, 128), dtype=float32)
('output size:', 262144, 

In [15]:
# feed_pc, feed_model_names, _ = all_pc_data.next_batch(100)
# feed_pc_tr = feed_pc[:, :n_pc_points]-EMD:
# feed_pc_te = feed_pc[:, -n_pc_points:]
# reconstructions = ae.reconstruct(feed_pc_tr)[0]
# latent_codes = ae.transform(feed_pc_tr)

In [16]:
# mmd_emd, mmd_cd, cov_emd, cov_cd = MMD_COV_EMD_CD(reconstructions, feed_pc_te, 
#                                                   32, normalize=normalize, verbose=True)
# print("MMD-EMD:%s"%mmd_emd)
# print("MMD-CD:%s"%mmd_cd)
# print("COV-EMD:%s"%cov_emd)
# print("COV-CD:%s"%cov_cd)

In [17]:
# mmd_emd, _ = minimum_mathing_distance(
#     reconstructions, feed_pc_te, 100, normalize=normalize, use_EMD=True)
# mmd_emd

In [18]:
# mmd_cd, _ = minimum_mathing_distance(
#     reconstructions, feed_pc_te, 100, normalize=normalize, use_EMD=False)
# mmd_cd

In [19]:
feed_pc, _, _ = all_pc_data.full_epoch_data()
feed_pc_tr_all = feed_pc[:, :n_pc_points]
feed_pc_te_all = feed_pc[:, -n_pc_points:]
print(feed_pc_tr_all.shape)
print(feed_pc_te_all.shape)

(405, 2048, 3)
(405, 2048, 3)


In [20]:
all_sample = []
all_ref = []
for i in range(feed_pc_tr_all.shape[0]):
    feed_pc_tr = feed_pc_tr_all[i:i+1]
    feed_pc_te = feed_pc_te_all[i:i+1]
    reconstructions = ae.reconstruct(feed_pc_tr)[0]
    all_sample.append(reconstructions)
    all_ref.append(feed_pc_te)

all_sample = np.concatenate(all_sample)
all_ref = np.concatenate(all_ref)
all_sample.shape, all_ref.shape

((405, 2048, 3), (405, 2048, 3))

In [23]:
mmd_emd, mmd_cd, _, _ = MMD_COV_EMD_CD(all_sample, all_ref, 
                                                  100, normalize=normalize, verbose=True)




MMD-COV Loop:   0%|          | 0/405 [00:00<?, ?it/s]

MMD-COV Loop:   0%|          | 1/405 [00:01<10:39,  1.58s/it]

MMD-COV Loop:   0%|          | 2/405 [00:03<10:33,  1.57s/it]

MMD-COV Loop:   1%|          | 3/405 [00:04<10:30,  1.57s/it]

MMD-COV Loop:   1%|          | 4/405 [00:06<10:27,  1.57s/it]

MMD-COV Loop:   1%|          | 5/405 [00:07<10:32,  1.58s/it]

MMD-COV Loop:   1%|▏         | 6/405 [00:09<10:20,  1.56s/it]

MMD-COV Loop:   2%|▏         | 7/405 [00:10<10:19,  1.56s/it]

MMD-COV Loop:   2%|▏         | 8/405 [00:12<10:13,  1.54s/it]

MMD-COV Loop:   2%|▏         | 9/405 [00:13<10:02,  1.52s/it]

MMD-COV Loop:   2%|▏         | 10/405 [00:15<10:07,  1.54s/it]

MMD-COV Loop:   3%|▎         | 11/405 [00:17<10:09,  1.55s/it]

MMD-COV Loop:   3%|▎         | 12/405 [00:18<10:03,  1.53s/it]

MMD-COV Loop:   3%|▎         | 13/405 [00:20<10:01,  1.54s/it]

MMD-COV Loop:   3%|▎         | 14/405 [00:21<09:58,  1.53s/it]

MMD-COV Loop:   4%|▎         | 15/405 [00:23<09:55,  1.5

MMD-COV Loop:  29%|██▉       | 117/405 [02:58<07:17,  1.52s/it]

MMD-COV Loop:  29%|██▉       | 118/405 [03:00<07:19,  1.53s/it]

MMD-COV Loop:  29%|██▉       | 119/405 [03:02<07:12,  1.51s/it]

MMD-COV Loop:  30%|██▉       | 120/405 [03:03<07:18,  1.54s/it]

MMD-COV Loop:  30%|██▉       | 121/405 [03:05<07:20,  1.55s/it]

MMD-COV Loop:  30%|███       | 122/405 [03:06<07:17,  1.55s/it]

MMD-COV Loop:  30%|███       | 123/405 [03:08<07:15,  1.55s/it]

MMD-COV Loop:  31%|███       | 124/405 [03:09<07:10,  1.53s/it]

MMD-COV Loop:  31%|███       | 125/405 [03:11<07:06,  1.52s/it]

MMD-COV Loop:  31%|███       | 126/405 [03:12<07:03,  1.52s/it]

MMD-COV Loop:  31%|███▏      | 127/405 [03:14<07:02,  1.52s/it]

MMD-COV Loop:  32%|███▏      | 128/405 [03:15<07:03,  1.53s/it]

MMD-COV Loop:  32%|███▏      | 129/405 [03:17<06:56,  1.51s/it]

MMD-COV Loop:  32%|███▏      | 130/405 [03:18<06:52,  1.50s/it]

MMD-COV Loop:  32%|███▏      | 131/405 [03:20<06:51,  1.50s/it]

MMD-COV Loop:  33%|███▎  

MMD-COV Loop:  57%|█████▋    | 232/405 [05:53<04:27,  1.55s/it]

MMD-COV Loop:  58%|█████▊    | 233/405 [05:55<04:26,  1.55s/it]

MMD-COV Loop:  58%|█████▊    | 234/405 [05:56<04:22,  1.53s/it]

MMD-COV Loop:  58%|█████▊    | 235/405 [05:58<04:19,  1.52s/it]

MMD-COV Loop:  58%|█████▊    | 236/405 [05:59<04:18,  1.53s/it]

MMD-COV Loop:  59%|█████▊    | 237/405 [06:01<04:15,  1.52s/it]

MMD-COV Loop:  59%|█████▉    | 238/405 [06:02<04:18,  1.55s/it]

MMD-COV Loop:  59%|█████▉    | 239/405 [06:04<04:16,  1.54s/it]

MMD-COV Loop:  59%|█████▉    | 240/405 [06:05<04:16,  1.56s/it]

MMD-COV Loop:  60%|█████▉    | 241/405 [06:07<04:15,  1.56s/it]

MMD-COV Loop:  60%|█████▉    | 242/405 [06:09<04:10,  1.54s/it]

MMD-COV Loop:  60%|██████    | 243/405 [06:10<04:10,  1.55s/it]

MMD-COV Loop:  60%|██████    | 244/405 [06:12<04:07,  1.54s/it]

MMD-COV Loop:  60%|██████    | 245/405 [06:13<04:04,  1.53s/it]

MMD-COV Loop:  61%|██████    | 246/405 [06:15<04:01,  1.52s/it]

MMD-COV Loop:  61%|██████

MMD-COV Loop:  86%|████████▌ | 347/405 [08:49<01:29,  1.55s/it]

MMD-COV Loop:  86%|████████▌ | 348/405 [08:51<01:27,  1.53s/it]

MMD-COV Loop:  86%|████████▌ | 349/405 [08:52<01:25,  1.52s/it]

MMD-COV Loop:  86%|████████▋ | 350/405 [08:54<01:24,  1.53s/it]

MMD-COV Loop:  87%|████████▋ | 351/405 [08:55<01:22,  1.52s/it]

MMD-COV Loop:  87%|████████▋ | 352/405 [08:57<01:20,  1.52s/it]

MMD-COV Loop:  87%|████████▋ | 353/405 [08:58<01:18,  1.51s/it]

MMD-COV Loop:  87%|████████▋ | 354/405 [09:00<01:17,  1.52s/it]

MMD-COV Loop:  88%|████████▊ | 355/405 [09:01<01:16,  1.52s/it]

MMD-COV Loop:  88%|████████▊ | 356/405 [09:03<01:14,  1.51s/it]

MMD-COV Loop:  88%|████████▊ | 357/405 [09:04<01:12,  1.51s/it]

MMD-COV Loop:  88%|████████▊ | 358/405 [09:06<01:08,  1.46s/it]

MMD-COV Loop:  89%|████████▊ | 359/405 [09:07<01:07,  1.47s/it]

MMD-COV Loop:  89%|████████▉ | 360/405 [09:09<01:08,  1.51s/it]

MMD-COV Loop:  89%|████████▉ | 361/405 [09:10<01:09,  1.57s/it]

MMD-COV Loop:  89%|██████

In [24]:
print("MMD-EMD:%s"%mmd_emd)
print("MMD-CD:%s"%mmd_cd)


MMD-EMD:0.29495442
MMD-CD:0.006728542


In [ ]:
# mmd_emd, _ = minimum_mathing_distance(
#     all_sample, all_ref, 200, normalize=normalize, use_EMD=True, verbose=True)
# mmd_emd

In [ ]:
# mmd_cd, _ = minimum_mathing_distance(
#     all_sample, all_ref, 50, normalize=normalize, use_EMD=False)
# mmd_cd